In [1]:
words = open('names.txt', 'r').read().splitlines()

In [2]:
import torch

In [3]:
N = torch.zeros((27,27), dtype = torch.int32)

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i , s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [5]:
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1, ix2] += 1

In [6]:
N[0]

tensor([   0, 4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591, 2422, 2963,
        1572, 2538, 1146,  394,  515,   92, 1639, 2055, 1308,   78,  376,  307,
         134,  535,  929], dtype=torch.int32)

In [7]:
p = N[0].float()
p = p / p.sum()
p

tensor([0.0000, 0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273,
        0.0184, 0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029,
        0.0512, 0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])

In [8]:
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]

'j'

In [10]:

P = (N+1).float()
P /= P.sum(1, keepdim= True)
# ahora estan todas las filas normalizadas

In [11]:
g = torch.Generator().manual_seed(2147483647)
for i in range(5):
    ix = 0
    word = []
    while True:
        #p = N[ix].float()
        #p = p / p.sum()
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement= True, generator = g).item()
        word.append(itos[ix])
        if ix == 0:
            break
    print(''.join(word))


junide.
janasah.
p.
cony.
a.


In [12]:
#ahora vamos a ver que tan bueno es el modelo, a partir de los nombres originales.
log_likelihood = 0.0
n = 0

for w in words[:10]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip (chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
        print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}') # la calidad del modelo, cuanto mas cerca de cero mejor.
print(f'{nll/n}') #es la calidad del modelo average

#¿que hace este codigo? primero que va por toda la grilla de probabilidades(que letra es probable que le siga a otra)
# despues con esa probabilidad, de agrega log, y luego ocn log_likelihood suma todas esas probabilidades
# y como la suma de los log, es lo mismo que la multiplicacion de su input, osea que log(a) + log(b) + log(c) = log(a*b*C)
# acordarse que a b y c son probabilidades, la log_likelihood que la multiplicacion de todas esas probabiliadaes.

# nada mas que su suma al final se le aplica un log por conveniencia.

.e: 0.0478 -3.0410
em: 0.0377 -3.2793
mm: 0.0253 -3.6753
ma: 0.3885 -0.9454
a.: 0.1958 -1.6305
.o: 0.0123 -4.3965
ol: 0.0779 -2.5526
li: 0.1774 -1.7293
iv: 0.0152 -4.1845
vi: 0.3508 -1.0476
ia: 0.1380 -1.9807
a.: 0.1958 -1.6305
.a: 0.1376 -1.9835
av: 0.0246 -3.7041
va: 0.2473 -1.3971
a.: 0.1958 -1.6305
.i: 0.0185 -3.9919
is: 0.0743 -2.5998
sa: 0.1478 -1.9119
ab: 0.0160 -4.1363
be: 0.2455 -1.4044
el: 0.1589 -1.8396
ll: 0.0962 -2.3408
la: 0.1876 -1.6733
a.: 0.1958 -1.6305
.s: 0.0641 -2.7468
so: 0.0654 -2.7270
op: 0.0121 -4.4180
ph: 0.1947 -1.6364
hi: 0.0955 -2.3485
ia: 0.1380 -1.9807
a.: 0.1958 -1.6305
.c: 0.0481 -3.0339
ch: 0.1869 -1.6774
ha: 0.2937 -1.2251
ar: 0.0963 -2.3405
rl: 0.0325 -3.4256
lo: 0.0496 -3.0047
ot: 0.0149 -4.2032
tt: 0.0670 -2.7031
te: 0.1281 -2.0549
e.: 0.1948 -1.6357
.m: 0.0792 -2.5358
mi: 0.1885 -1.6687
ia: 0.1380 -1.9807
a.: 0.1958 -1.6305
.a: 0.1376 -1.9835
am: 0.0482 -3.0321
me: 0.1228 -2.0971
el: 0.1589 -1.8396
li: 0.1774 -1.7293
ia: 0.1380 -1.9807
a.: 0.1958 -

In [13]:
# GOAL : maximize likelihood of the data w.r.t model parameters (statistical modeling)
# equivalent to maximizeing the log likelihood (because log in monotonic) monotonic is a funtion that preserves the given order
# equivalent to minimizing the negative log likelihood
# equivalent to minimizing the average negative log likelihood

# acordarse que log(a) + log(b) + log(c) = log(a*b*b)



In [14]:
# nuestra red neuronal, va a recibir un caracter, y va a devolver la probabilidad del SIGUIENTE caracter.
# y vamos a usar la loss-funtion, osea, nos vamos a basar en -log_likelihood, conla loss-funtion, vamos a buscar un minimo
# y para buscar un minimo vamos a jugar con los weights

In [15]:
# create the training set of all the bigrams

xs, ys = [], [] # vamos a hacer dos listas, 'xs' son los inputs, y 'ys' son los objetivos
# en el primero van a estar los caracteres, y en el segundo la prediccion

for w in words[:1]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    print(ch1,ch2)
    xs.append(ix1)
    ys.append(ix2)

# y ahora nos conviene usar tensores

xs = torch.tensor(xs)
yx = torch.tensor(ys)

# entonces lo que hace esto es lo mismo que hacíamos antes, darle un valor de 0 a 26 a las letras, pero esta vez usamos tensores
# luego usamos la inteligente funcion zip, para ordenar la palabra de forma que sea un bigram con
# .esteban 
# esteban. entonces zip, va a ir agarrando (., e), (e, s) (s,t) y así hasta (n , .)
type(xs[0].item())
P[0]

. e
e m
m m
m a
a .


tensor([3.1192e-05, 1.3759e-01, 4.0767e-02, 4.8129e-02, 5.2745e-02, 4.7785e-02,
        1.3038e-02, 2.0898e-02, 2.7293e-02, 1.8465e-02, 7.5577e-02, 9.2452e-02,
        4.9064e-02, 7.9195e-02, 3.5777e-02, 1.2321e-02, 1.6095e-02, 2.9008e-03,
        5.1154e-02, 6.4130e-02, 4.0830e-02, 2.4641e-03, 1.1759e-02, 9.6070e-03,
        4.2109e-03, 1.6719e-02, 2.9008e-02])

In [16]:
# ahora vamos a usar una funtion llamada ONE HOT, la cual para un numero, crea un vector, que tiene longitud numero

In [17]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes =27).float() # no se puede usar dtype =float32 porque en la especificacion de torch no deja.
# esto lo que haces es hacer una grilla, donde las filas son las posiciones y estan llenas de 0 exepto en la columna donde esta el valor de la posicion.
 # en la grilla, la posicion [2,13] es un 1, ya que en la palabra emma, la letra m xtoi(m) = 13
# hay que tener mucho cuidado con los data types de xenc

In [19]:
W = torch.randn((27,27)) # crea un tensor con numeros random de la normal distribution, muchos van a ser 0 y algunos van a llegar a ser 3
xenc @ W # el @ es la multiplicacion de matrices

tensor([[ 0.3984, -0.7719, -0.5301, -0.3948, -1.5728, -1.1998,  1.1549,  1.3326,
          0.3140,  0.4446, -0.4605, -1.5793,  0.2157,  1.7033,  1.9903, -0.5601,
         -1.3735, -1.4781,  0.8445,  0.2813, -0.9190, -0.2010,  0.5124,  1.0930,
         -2.1139, -0.2706, -1.2021],
        [-0.7743,  2.4054, -1.0919, -0.1741, -0.6260,  0.8478, -1.1761, -0.0519,
         -1.2176,  0.6090, -0.8973,  0.9077,  1.5546, -0.1561,  0.2285, -1.3915,
          1.1705, -1.7693, -0.0811, -0.5382,  0.6206, -0.5555,  0.5031,  0.3340,
          1.6046, -0.5954,  0.2818],
        [ 1.6366,  0.2133, -0.9695, -0.2006,  0.7623, -0.1863, -0.2994, -0.9337,
          0.0239,  0.0599,  0.2794, -1.4746, -0.4258,  0.4068,  0.6020,  0.1340,
         -0.2579,  0.6377,  0.0125, -1.4132,  1.4148,  0.8470,  1.6648, -0.6872,
          0.3984, -0.8069, -1.0935],
        [ 1.6366,  0.2133, -0.9695, -0.2006,  0.7623, -0.1863, -0.2994, -0.9337,
          0.0239,  0.0599,  0.2794, -1.4746, -0.4258,  0.4068,  0.6020,  0.1340

In [ ]:
(xenc @ W) 